In [4]:
import pandas as pd
import os

summary = '/home/truthling/MGGen/Escherichia_coli_assembly.txt'
genome_files = '/home/truthling/MGGen/Acinetobacter_nosocomialis/genome_files/'
df = pd.read_csv(summary, delimiter='\t', index_col=0)

**Check for and download new files**
- instead of getting the accession number from the actual filenames, get from a log file
- instead of removing files that fail quality control, move them to a different folder

In [ ]:
test_summary = pd.read_csv('test_summary.txt', delimiter='\t', index_col=0)

accession_list = [] # list of files we do have

for f in os.listdir(genome_files):
    accession = (f.split('_')[0:2])
    accession = ('_'.join(accession))
    accession_list.append(accession)

new_files = []
    
for row in test_summary.index:
    if row in accession_list:
#         print("already have " + row)
        continue
    new_files.append(row)

new_ftps = open('new_ftps.txt', 'w')

new_urls = []

for new in new_files:
    if new in test_summary.index:
        new_url = test_summary.get_value(new, 'ftp_path')
        new_urls.append(new_url)

for url in new_urls:
    new_ftps.write('%s\n' % url)

```bash
cat new_ftps.txt | \
sed -r 's|(ftp://ftp.ncbi.nlm.nih.gov/genomes/all/)(GCA_.+)|\1\2/\2_genomic.fna.gz|'>new_genome_urls.txt
wget -P ./genome_files/ --input genome_urls.txt
gunzip ./genome_files/*.gz
```

- If both `infraspecific_name` and `isolate` are empty, fill `infraspecific_name` with "NA"
- If just `infraspecific_name` is empty, fill with the value for `isolate`
- Replace unwanted characters

In [5]:
df

,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path
# assembly_accession,,,,,,,,,,,,,,,,,,,
GCA_000005845.2,PRJNA225,SAMN02604091,NaN,reference genome,511145,562,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,NaN,latest,Complete Genome,Major,Full,2013/09/26,ASM584v2,Univ. Wisconsin,GCF_000005845.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000006665.1,PRJNA259,SAMN02604092,NaN,na,155864,562,Escherichia coli O157:H7 str. EDL933,strain=EDL933,NaN,latest,Chromosome,Major,Full,2004/12/06,ASM666v1,NaN,GCF_000006665.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000007445.1,PRJNA313,SAMN02604094,NaN,na,199310,562,Escherichia coli CFT073,strain=CFT073,NaN,latest,Complete Genome,Major,Full,2002/12/06,ASM744v1,NaN,GCF_000007445.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000008865.1,PRJNA226,SAMN01911278,NaN,reference genome,386585,562,Escherichia coli O157:H7 str. Sakai,strain=Sakai substr. RIMD 0509952,NaN,latest,Complete Genome,Major,Full,2004/05/11,ASM886v1,NaN,GCF_000008865.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000009565.2,PRJEA28965,SAMEA2272558,NaN,na,469008,562,Escherichia coli BL21(DE3),strain=BL21(DE3),NaN,latest,Complete Genome,Major,Full,2010/12/09,ASM956v1,NaN,GCF_000009565.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000010245.1,PRJNA16351,NaN,NaN,na,316407,562,Escherichia coli str. K-12 substr. W3110,strain=K-12 substr. W3110,NaN,latest,Complete Genome,Major,Full,2006/01/23,ASM1024v1,NaN,GCF_000010245.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000010385.1,PRJNA18057,NaN,NaN,na,409438,562,Escherichia coli SE11,strain=SE11,NaN,latest,Complete Genome,Major,Full,2008/10/22,ASM1038v1,NaN,GCF_000010385.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000010485.1,PRJDA19053,NaN,NaN,na,431946,562,Escherichia coli SE15,strain=SE15,NaN,latest,Complete Genome,Major,Full,2009/12/14,ASM1048v1,NaN,GCF_000010485.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000010745.1,PRJDA32511,NaN,NaN,na,585395,562,Escherichia coli O103:H2 str. 12009,strain=12009,NaN,latest,Complete Genome,Major,Full,2009/09/10,ASM1074v1,NaN,GCF_000010745.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...


In [3]:
df.update(df['infraspecific_name'][(df['infraspecific_name'].isnull()) & (df['isolate'].isnull())].fillna('NA'))
df.update(df['infraspecific_name'][(df['infraspecific_name'].isnull()) & (df['isolate'].notnull())].fillna(df['isolate']))
negate_list = ' '
df.organism_name.replace({'[^\w'+negate_list+']': ''}, regex=True, inplace=True)
df.infraspecific_name.replace({'[ =]': '_'}, regex=True, inplace=True)
df.assembly_level.replace({' ': '_'}, regex=True, inplace=True)
df

,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path
# assembly_accession,,,,,,,,,,,,,,,,,,,
GCA_000010525.1,PRJDA19267,NaN,NaN,representative genome,438753,7,Azorhizobium caulinodans ORS 571,strain_ORS_571,NaN,latest,Complete_Genome,Major,Full,2007/10/16,ASM1052v1,NaN,GCF_000010525.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000007365.1,PRJNA312,SAMN02604269,NaN,representative genome,198804,9,Buchnera aphidicola str Sg Schizaphis graminum,strain_Sg,NaN,latest,Complete_Genome,Major,Full,2002/07/02,ASM736v1,NaN,GCF_000007365.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000007725.1,PRJNA256,SAMN02604289,NaN,representative genome,224915,9,Buchnera aphidicola str Bp Baizongia pistaciae,strain_Bp_(Baizongia_pistaciae),NaN,latest,Complete_Genome,Major,Full,2003/01/29,ASM772v1,NaN,GCF_000007725.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000009605.1,PRJNA245,NaN,NaN,reference genome,107806,9,Buchnera aphidicola str APS Acyrthosiphon pisum,Tokyo1998,Tokyo1998,latest,Complete_Genome,Major,Full,2004/05/11,ASM960v1,NaN,GCF_000009605.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000021065.1,PRJNA31223,SAMN02604133,NaN,na,561501,9,Buchnera aphidicola str Tuc7 Acyrthosiphon pisum,strain_Tuc7,NaN,latest,Complete_Genome,Major,Full,2009/01/06,ASM2106v1,NaN,GCF_000021065.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000021085.1,PRJNA31225,SAMN02604134,NaN,na,563178,9,Buchnera aphidicola str 5A Acyrthosiphon pisum,strain_5A,NaN,latest,Complete_Genome,Major,Full,2009/01/06,ASM2108v1,NaN,GCF_000021085.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000090965.1,PRJNA16372,SAMN02604290,NaN,representative genome,372461,9,Buchnera aphidicola BCc,strain_BCc,NaN,latest,Complete_Genome,Major,Full,2006/10/18,ASM9096v1,Valencia Univ.,GCF_000090965.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000174075.1,PRJNA33811,SAMN02470818,ACFK00000000.1,na,593275,9,Buchnera aphidicola str LSR1 Acyrthosiphon pisum,strain_LSR1_(Acyrthosiphon_pisum),NaN,latest,Complete_Genome,Major,Full,2009/03/02,ASM17407v1,NaN,GCF_000174075.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...
GCA_000183225.1,PRJNA43505,SAMN02604265,NaN,na,713603,9,Buchnera aphidicola str LL01 Acyrthosiphon pisum,strain_LL01,NaN,latest,Complete_Genome,Major,Full,2010/11/04,ASM18322v1,NaN,GCF_000183225.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA_000...


**Rename files**

In [ ]:
for f in os.listdir("/home/truthling/MGGen/renamed/"):
    if f.startswith('GCA'):
        id = (f.split('_')[0:2])
        id = ('_'.join(id))
        
    for row in df.index:
        if id == row:
            org_name = df.get_value(id, 'organism_name')
            strain = df.get_value(id, 'infraspecific_name').strip('strain=')
            assembly_level = df.get_value(id, 'assembly_level')
            newname = '{}_{}_{}_{}.fasta'.format(id, org_name, strain, assembly_level)
            os.rename(f, newname)

In [ ]:
list = []
newdir = "/home/truthling/MGGen/renamed/"
for root, dirs, filenames in os.walk('/home/truthling/MGGen/renamed/'):
    for f in filenames:
#         root = root+"/"
        old = os.path.join(root,filename)
#         print(old)

        if f.startswith('GCA'):
            id = (f.split('_')[0:2])
            id = ('_'.join(id))

        for row in df.index:
            if id == row:
                org_name = df.get_value(id, 'organism_name')
                strain = df.get_value(id, 'infraspecific_name').strip('strain=')
                assembly_level = df.get_value(id, 'assembly_level')
                newname = '{}_{}_{}_{}.fasta'.format(id, org_name, strain, assembly_level)
                print(old)
#                 os.rename(old, newname)
#                 print(newdir+newname)
#                 os.system("cp %s %s/%s" % (old, newdir, newname))
#                 print(root + newname)
#                 print(old)
#                 print(newdir + newname +"\n\n")
#                 list.append(old)
#                 print("%s\n\n %s/%s" % (old, newdir, newname))

# len(list)

In [ ]:
summary = '/home/truthling/MGGen/renamed/assembly_summary.txt'
genome_files = '/home/truthling/MGGen/Acinetobacter_nosocomialis/genome_files/'
df = pd.read_csv(summary, delimiter='\t', index_col=0)
copiedfiles = '/home/truthling/MGGen/renamed/'

for f in os.listdir(copiedfiles):
    if f.startswith('GCA'):
        id = (f.split('_')[0:2])
        id = ('_'.join(id))

    for row in df.index:
        if id == row:
            org_name = df.get_value(id, 'organism_name')
            strain = df.get_value(id, 'infraspecific_name').strip('strain=')
            assembly_level  = df.get_value(id, 'assembly_level')
            newname = '{}_{}_{}_{}.fasta'.format(id, org_name, strain, assembly_level)
            os.rename(copiedfiles+f, copiedfiles+newname)